In [85]:
# merge all the files with the affix ".xlsx" in the current folder into one file

import pandas as pd
import os


In [87]:
# get all the files with the affix ".xlsx" in the current folder
files = [file for file in os.listdir() if file.endswith(".xlsx")]

# read all the files into a list of dataframes
dfs = [pd.read_excel(file) for file in files]

# merge all the dataframes into one dataframe
df = pd.concat(dfs)

# write the merged dataframe into a file
df.to_excel("test.xlsx", index=False)

In [96]:
def aggregate_abstracts(FILE_NAME):
    # read the merged dataframe
    df = pd.read_excel(FILE_NAME)
    # print the number of rows in the merged dataframe
    print(len(df))
    
    print(df.columns)
    print(df.head())
    if len(df.columns) == 9:
        df.columns = ['article_id', 'abstract', 'Background', 'Purpose', 'Method', 'Result', 'Conclusions', 'Limitation', 'Others']
    if len(df.columns) == 14:
        df.columns = ['dataset_type', 'row_id', 'article_id', 'category', 'abstract', 'Background', 'Purpose', 'Method', 'Result', 'Conclusions', 'Limitation', 'Others',  'Errors', 'TooLong']
        assert len(df.columns) == 14

    
    abstracts = {}
    facets_for_abstracts = {}
    for row in df.iterrows():
        # split the abstract into sentences according to the tages <S0>, <S1>, <S2>, ...
        sentences =  row[1]['abstract'].split("<S")[1:]        
        sentences = [sentence[sentence.find(">")+1:] for sentence in sentences] # get the content of each sentence
        sentences = [sentence[:sentence.find("</S")] for sentence in sentences] # remove the tags </S0>, </S1>, </S2>, ...

        # extract facet for each sentence
        facets = ['Others'] * len(sentences)
        for i in range(len(sentences)):
            identifier = 'S'+str(i)
            for tag in [ 'Background', 'Purpose', 'Method', 'Result', 'Conclusions', 'Limitation', 'Others']:
                sent_ids = row[1][tag]
                if type(sent_ids) == float:
                    continue
                if identifier in sent_ids:
                    facets[i] = tag
                    break

        abstracts[str(row[1]['article_id'])] = sentences
        facets_for_abstracts[str(row[1]['article_id'])] = facets
    return abstracts, facets_for_abstracts


abstracts, facets_for_abstracts = aggregate_abstracts("arxiv_computer-science.xlsx")

500
Index(['dataset_type', 'row_id', 'arxiv_id', 'category', 'abstract',
       'Introduction/Background', 'Aim/Purpose', 'Methodology',
       'Results / Contributions ', 'Conclusions',
       'Limitations of Research / Gap', 'Not Sure', 'ERRORS', 'TOO LONG'],
      dtype='object')
  dataset_type  row_id    arxiv_id category  \
0        train   44884   1408.1268       cs   
1        train   52749   1409.5561       cs   
2        train   48309   cs0406058       cs   
3        train  134998  1703.07247       cs   
4        train   47466   cs0408044       cs   

                                            abstract Introduction/Background  \
0  <S0> In this paper we develop an algorithm for...                      S0   
1  <S0> We consider a network design problem call...                      S0   
2  <S0> We present a protocol for verification of...                      S0   
3  <S0> In the tree augmentation problem we are g...                S0,S1,S3   
4  <S0> Flux is a programming met

ValueError: Length mismatch: Expected axis has 14 elements, new values have 9 elements

In [98]:
# save to json file
import json
with open('test_abstracts.json', 'w') as fp:
    json.dump(abstracts, fp)

with open('test_facets.json', 'w') as fp:
    json.dump(facets_for_abstracts, fp)